In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from datetime import timedelta
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)  # Show all columns


# Function to convert MM:SS to seconds, handling NaN values
def mmss_to_seconds(mmss):
    mmss = str(mmss)  # Ensure mmss is a string
    if pd.isna(mmss):
        return np.nan  # Return NaN if the value is NaN
    else:
        minutes, seconds = map(int, mmss.split(':'))
        return minutes * 60 + seconds

# Function to convert total seconds to MM:SS format
def mmss_to_seconds(mmss):
    try:
        if isinstance(mmss, str) and ':' in mmss:
            minutes, seconds = map(int, mmss.split(':'))
            return minutes * 60 + seconds
        elif mmss == '0' or mmss == 0 or pd.isna(mmss):
            return 0
    except Exception:
        pass
    return 0  # fallback if bad format

In [12]:
player_data

,playerId,isActive,currentTeamId,teamPlaceNameWithPreposition,firstName,lastName,badgesLogos,badgesNames,sweaterNumber,position,headshot,heroImage,heightInInches,heightInCentimeters,weightInPounds,weightInKilograms,birthDate,birthCity,birthStateProvince,birthCountry,shootsCatches,draftYear,draftTeam,draftRound,draftPickInRound,draftOverall,playerSlug,inTop100AllTime,inHHOF,featuredSeason,featuredSeasonAssists,featuredSeasonGWG,featuredSeasonGP,featuredSeasonGoals,featuredSeasonOTGoals,featuredSeasonPIM,featuredSeasonPlusMinus,featuredSeasonPts,featuredSeasonPPG,featuredSeasonPPPoints,featuredSeasonShootingPct,featuredSeasonSHG,featuredSeasonSHPts,featuredSeasonShots,regSeasonCareerAssists,regSeasonCareerGWG,regSeasonCareerGP,regSeasonCareerGoals,regSeasonCareerOTGoals,regSeasonCareerPIM,regSeasonCareerPlusMinus,regSeasonCareerPts,regSeasonCareerPPG,regSeasonCareerPPPoints,regSeasonCareerShootingPct,regSeasonCareerSHG,regSeasonCareerSHPts,regSeasonCareerShots,playoffsCareerAssists,playoffsCareerGWG,playoffsCareerGP,playoffsCareerGoals,playoffsCareerOTGoals,playoffsCareerPIM,playoffsCareerPlusMinus,playoffsCareerPts,playoffsCareerPPG,playoffsCareerPPPoints,playoffsCareerShootingPct,playoffsCareerSHG,playoffsCareerSHPts,playoffsCareerShots,last5Games,seasonTotals,awardNames,awardSeasons,currentTeamRoster,featuredSeasonGAA,featuredSeasonLosses,featuredSeasonSO,featuredSeasonTies,featuredSeasonWins,featuredSeasonGS,featuredSeasonGA,featuredSeasonTOI,regSeasonCareerGAA,regSeasonCareerLosses,regSeasonCareerSO,regSeasonCareerTies,regSeasonCareerWins,regSeasonCareerGS,regSeasonCareerGA,regSeasonCareerTOI,playoffsCareerGAA,playoffsCareerLosses,playoffsCareerSO,playoffsCareerTies,playoffsCareerWins,playoffsCareerGS,playoffsCareerGA,playoffsCareerTOI,featuredSeasonSavePct,featuredSeasonOTLosses,featuredSeasonShotsAgainst,regSeasonCareerSavePct,regSeasonCareerOTLosses,regSeasonCareerShotsAgainst,playoffsCareerSavePct,playoffsCareerOTLosses,playoffsCareerShotsAgainst
0,0,isActive,currentTeamId,teamPlaceNameWithPreposition,firstName,lastName,badgesLogos,badgesNames,sweaterNumber,position,headshot,heroImage,heightInInches,heightInCentimeters,weightInPounds,weightInKilograms,birthDate,birthCity,birthStateProvince,birthCountry,shootsCatches,draftYear,draftTeam,draftRound,draftPickInRound,draftOverall,playerSlug,inTop100AllTime,inHHOF,featuredSeason,featuredSeasonAssists,featuredSeasonGWG,featuredSeasonGP,featuredSeasonGoals,featuredSeasonOTGoals,featuredSeasonPIM,featuredSeasonPlusMinus,featuredSeasonPts,featuredSeasonPPG,featuredSeasonPPPoints,featuredSeasonShootingPct,featuredSeasonSHG,featuredSeasonSHPts,featuredSeasonShots,regSeasonCareerAssists,regSeasonCareerGWG,regSeasonCareerGP,regSeasonCareerGoals,regSeasonCareerOTGoals,regSeasonCareerPIM,regSeasonCareerPlusMinus,regSeasonCareerPts,regSeasonCareerPPG,regSeasonCareerPPPoints,regSeasonCareerShootingPct,regSeasonCareerSHG,regSeasonCareerSHPts,regSeasonCareerShots,playoffsCareerAssists,playoffsCareerGWG,playoffsCareerGP,playoffsCareerGoals,playoffsCareerOTGoals,playoffsCareerPIM,playoffsCareerPlusMinus,playoffsCareerPts,playoffsCareerPPG,playoffsCareerPPPoints,playoffsCareerShootingPct,playoffsCareerSHG,playoffsCareerSHPts,playoffsCareerShots,last5Games,seasonTotals,awardNames,awardSeasons,currentTeamRoster,featuredSe,featuredSe,featuredSe,featuredSe,featuredSe,featuredSe,featuredSe,featuredSe,regSeasonC,regSeasonC,regSeasonC,regSeasonC,regSeasonC,regSeasonC,regSeasonC,regSeasonC,playoffsCa,playoffsCa,playoffsCa,playoffsCa,playoffsCa,playoffsCa,playoffsCa,playoffsCa,featuredSe,featuredSe,featuredSe,regSeasonC,regSeasonC,regSeasonC,playoffsCa,playoffsCa,playoffsCa
1,8444850,False,0,0,Henry,Harris,0,0,12,R,https://assets.nhle.com/mugs/nhl/latest/844485...,https://assets.nhle.com/mugs/actionshots/1296x...,71,180,185,84,1906-04-28,Kenora,Ontario,CAN,L,0,0,0,0,0,henry-harris-8444850,0,0,19301931,4,0,32,2,0,20,0,6,0,0,0,0,0,0,4,0,32,2,0,20,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"[{'assists'

In [ ]:
# load player data
player_data = pd.read_csv("C:/users/conno/OneDrive/Documents/Personal-and-NHL-Website/public_html/resources/data/nhl_players.csv")
player_data = player_data.fillna(0)

# Load your shot data
pbp_data = pd.read_csv("C:/users/conno/OneDrive/Documents/Personal-and-NHL-Website/public_html/resources/data/nhl_plays.csv")
# print(data.head(10))
pbp_data = pbp_data.fillna(0).sort_values(by=['gameID', 'eventID'])

# merge player and goalie stats into play by play data - need to do it sequentially
# Merge shooting player stats into pbp_data
# pbp_with_shooter = pbp_data.merge(
#     player_data,
#     how='left',
#     left_on='shootingPlayerId',
#     right_on='playerId',
#     suffixes=('', '_shooter')
# )

pbp_w_shooter_data = pd.merge(pbp_data, player_data[['playerId', 'shootsCatches', 'draftOverall', 'regSeasonCareerShootingPct']], how='left', left_on='shootingPlayerId', right_on='playerId', suffixes=('', '_shooter'))

# Merge goalie stats into the result
# pbp = pbp_with_shooter.merge(
#     player_data,
#     how='left',
#     left_on='goalieInNetId',
#     right_on='playerId',
#     suffixes=('', '_goalie')
# )

pbp_full = pd.merge(pbp_w_shooter_data, player_data[['playerId', 'shootsCatches', 'draftOverall', 'regSeasonCareerSavePct']], how='left', left_on='goalieInNetId', right_on='playerId', suffixes=('', '_goalie'))


<ipython-input-18-3b01072f3924>:2: DtypeWarning: Columns (1,2,8,12,13,14,15,21,23,24,25,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,93,94,95,96,97,98,99,101,102,103,104,105,106,107,108,109) have mixed types. Specify dtype option on import or set low_memory=False.
  player_data = pd.read_csv("C:/users/conno/OneDrive/Documents/Personal-and-NHL-Website/public_html/resources/data/nhl_players.csv")
<ipython-input-18-3b01072f3924>:6: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  pbp_data = pd.read_csv("C:/users/conno/OneDrive/Documents/Personal-and-NHL-Website/public_html/resources/data/nhl_plays.csv")


KeyError: "['shootsCatches_goalie', 'draftOverall_goalie'] not in index"

In [ ]:
pbp_full

,gameID,eventID,period,timeInPeriod,timeRemaining,situationCode,typeCode,typeDescKey,xCoord,yCoord,zoneCode,eventOwnerTeamId,faceoffLoserId,faceoffWinnerId,hittingPlayerId,hitteePlayerId,shotType,shootingPlayerId,goalieInNetId,awaySOG,homeSOG,reason,takeawayGiveawayPlayerId,blockingPlayerId,scoringPlayerId,assist1PlayerId,assist1PlayerTotal,awayScore,homeScore,penaltySeverity,penaltyType,duration,committerId,drawerId,homeTeamDefendingSide
0,2001021074,10634706,1,12:25:00,07:35:00,0,506,shot-on-goal,0,0,0,14,0,0,0,0,slap,8466162,8455994,2,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
1,2001021074,10634707,1,14:21:00,05:39:00,0,506,shot-on-goal,0,0,0,6,0,0,0,0,0,8466145,8459140,3,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
2,2001021074,10634708,1,14:45:00,05:15:00,0,506,shot-on-goal,0,0,0,6,0,0,0,0,wrist,8466145,8459140,4,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
3,2001021074,10634709,1,15:34:00,04:26:00,0,506,shot-on-goal,0,0,0,6,0,0,0,0,wrist,8456464,8459140,5,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
4,2001021074,10634710,1,17:25:00,02:35:00,0,506,shot-on-goal,0,0,0,6,0,0,0,0,slap,8466035,8459140,6,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1094983,2010020288,214,1,19:29:00,00:31:00,1551,503,hit,-61,-40,0,18,0,0,8470642,8466035,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
1094984,2010020288,215,1,19:53:00,00:07:00,1551,516,stoppage,0,0,0,0,0,0,0,0,0,8465210,0,0,0,goalie-stopped-after-sog,0,0,0,0,0,0,0,0,0,0,0,0,0.0
1094985,2010020288,216,1,19:55:00,00:05:00,1551,508,blocked-shot,-55,-10,0,12,0,0,0,0,wrist,0,0,0,0,0,0,8469547,0,0,0,0,0,0,0,0,0,0,0.0
1094986,2010020288,217,1,19:56:00,00:04:00,1551,502,faceoff,-69,-22,0,12,8468162,8474091,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0


In [3]:
pbp

,gameID,eventID,period,timeInPeriod,timeRemaining,situationCode,typeCode,typeDescKey,xCoord,yCoord,zoneCode,eventOwnerTeamId,faceoffLoserId,faceoffWinnerId,hittingPlayerId,hitteePlayerId,shotType,shootingPlayerId,goalieInNetId,awaySOG,homeSOG,reason,takeawayGiveawayPlayerId,blockingPlayerId,scoringPlayerId,assist1PlayerId,assist1PlayerTotal,awayScore,homeScore,penaltySeverity,penaltyType,duration,committerId,drawerId,homeTeamDefendingSide,playerId,isActive,currentTeamId,teamPlaceNameWithPreposition,firstName,lastName,badgesLogos,badgesNames,sweaterNumber,position,headshot,heroImage,heightInInches,heightInCentimeters,weightInPounds,weightInKilograms,birthDate,birthCity,birthStateProvince,birthCountry,shootsCatches,draftYear,draftTeam,draftRound,draftPickInRound,draftOverall,playerSlug,inTop100AllTime,inHHOF,featuredSeason,featuredSeasonAssists,featuredSeasonGWG,featuredSeasonGP,featuredSeasonGoals,featuredSeasonOTGoals,featuredSeasonPIM,featuredSeasonPlusMinus,featuredSeasonPts,featuredSeasonPPG,featuredSeasonPPPoints,featuredSeasonShootingPct,featuredSeasonSHG,featuredSeasonSHPts,featuredSeasonShots,regSeasonCareerAssists,regSeasonCareerGWG,regSeasonCareerGP,regSeasonCareerGoals,regSeasonCareerOTGoals,regSeasonCareerPIM,regSeasonCareerPlusMinus,regSeasonCareerPts,regSeasonCareerPPG,regSeasonCareerPPPoints,regSeasonCareerShootingPct,regSeasonCareerSHG,regSeasonCareerSHPts,regSeasonCareerShots,playoffsCareerAssists,playoffsCareerGWG,playoffsCareerGP,playoffsCareerGoals,playoffsCareerOTGoals,playoffsCareerPIM,playoffsCareerPlusMinus,playoffsCareerPts,playoffsCareerPPG,playoffsCareerPPPoints,playoffsCareerShootingPct,playoffsCareerSHG,playoffsCareerSHPts,playoffsCareerShots,last5Games,seasonTotals,awardNames,awardSeasons,currentTeamRoster,featuredSeasonGAA,featuredSeasonLosses,featuredSeasonSO,featuredSeasonTies,featuredSeasonWins,featuredSeasonGS,featuredSeasonGA,featuredSeasonTOI,regSeasonCareerGAA,regSeasonCareerLosses,regSeasonCareerSO,regSeasonCareerTies,regSeasonCareerWins,regSeasonCareerGS,regSeasonCareerGA,regSeasonCareerTOI,playoffsCareerGAA,playoffsCareerLosses,playoffsCareerSO,playoffsCareerTies,playoffsCareerWins,playoffsCareerGS,playoffsCareerGA,playoffsCareerTOI,featuredSeasonSavePct,featuredSeasonOTLosses,featuredSeasonShotsAgainst,regSeasonCareerSavePct,regSeasonCareerOTLosses,regSeasonCareerShotsAgainst,playoffsCareerSavePct,playoffsCareerOTLosses,playoffsCareerShotsAgainst,playerId_goalie,isActive_goalie,currentTeamId_goalie,teamPlaceNameWithPreposition_goalie,firstName_goalie,lastName_goalie,badgesLogos_goalie,badgesNames_goalie,sweaterNumber_goalie,position_goalie,headshot_goalie,heroImage_goalie,heightInInches_goalie,heightInCentimeters_goalie,weightInPounds_goalie,weightInKilograms_goalie,birthDate_goalie,birthCity_goalie,birthStateProvince_goalie,birthCountry_goalie,shootsCatches_goalie,draftYear_goalie,draftTeam_goalie,draftRound_goalie,draftPickInRound_goalie,draftOverall_goalie,playerSlug_goalie,inTop100AllTime_goalie,inHHOF_goalie,featuredSeason_goalie,featuredSeasonAssists_goalie,featuredSeasonGWG_goalie,featuredSeasonGP_goalie,featuredSeasonGoals_goalie,featuredSeasonOTGoals_goalie,featuredSeasonPIM_goalie,featuredSeasonPlusMinus_goalie,featuredSeasonPts_goalie,featuredSeasonPPG_goalie,featuredSeasonPPPoints_goalie,featuredSeasonShootingPct_goalie,featuredSeasonSHG_goalie,featuredSeasonSHPts_goalie,featuredSeasonShots_goalie,regSeasonCareerAssists_goalie,regSeasonCareerGWG_goalie,regSeasonCareerGP_goalie,regSeasonCareerGoals_goalie,regSeasonCareerOTGoals_goalie,regSeasonCareerPIM_goalie,regSeasonCareerPlusMinus_goalie,regSeasonCareerPts_goalie,regSeasonCareerPPG_goalie,regSeasonCareerPPPoints_goalie,regSeasonCareerShootingPct_goalie,regSeasonCareerSHG_goalie,regSeasonCareerSHPts_goalie,regSeasonCareerShots_goalie,playoffsCareerAssists_goalie,playoffsCareerGWG_goalie,playoffsCareerGP_goalie,playoffsCareerGoals_goalie,playoffsCareerOTGoals_goalie,playoffsCareerPIM_goalie,playoffsCareerPlusMinus_goalie,pl